In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, random_state=0, shuffle=False)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

In [ ]:
clf.fit(X,y)

In [ ]:
print(clf.feature_importances_)

In [ ]:
print(clf.predict([[0, 0, 0, 0]]))

In [54]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
X, y = make_regression(n_features=4, n_informative=2, random_state=0,shuffle=False)

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)

In [ ]:
regr.fit(X,y)
print(regr.predict([[0,0,0,0]]))

In [1]:
import numpy as np

In [4]:
N = 1000
p = 100
def genXY(N, p):
    X = np.random.rand(N*p).reshape(N, p)
    y = 10 * np.exp(-2*np.sum(X[:,0:5]**2, axis = 1)) + np.sum(X[:,5:35], axis = 1) + np.random.normal(0, 1.3, N)
    return X, y

In [58]:
X, y = genXY(1000,100)

In [7]:
from sklearn.ensemble import RandomForestRegressor

In [8]:
rf = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=2500)
gbm1 = GradientBoostingRegressor(n_estimators=2500, learning_rate=0.01, subsample=1)
gbm1.fit(X, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=2500, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [9]:
rf.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=2500, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [10]:
def mse(regr):
    X_test, y_test = genXY(500, 100)
    y_pred = regr.predict(X_test)
    return np.mean((y_pred - y_test)**2)

In [11]:
mse(rf)

5.26315845952546

In [2]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
res = np.empty([4, 5])
for i in range(5):
    gbm1 = GradientBoostingRegressor(n_estimators=500*(1+i), learning_rate=0.01, subsample=1)
    gbm1.fit(X, y)
    gbm2 = GradientBoostingRegressor(n_estimators=500*(1+i), learning_rate=0.01, subsample=0.1)
    gbm2.fit(X, y)
    bs1, lasso1 = ISLE(X, y, model = "GB", n_est = 500*(1+i))
    bs2, lasso2 = ISLE(X, y, model = "RF", n_est = 500*(1+i))
    res[0, i] = mse(gbm1)
    res[1, i] = mse(gbm2)
    res[2, i] = mse_isle(bs1, lasso1)
    res[3, i] = mse_isle(bs2, lasso2)

In [55]:
def ISLE(X, y, eta = 0.5, nu = 0.1, M = 10, model = "GB", n_est = 500):
    N = len(X)
    # step 1
    f_hat = np.repeat(np.mean(y), N)
    bs = [None] * M
    for m in range(M):
        Sm_idx = np.random.choice(N, int(eta*N), replace = False)
        if model == "GB":
            bs[m] = GradientBoostingRegressor(n_estimators=n_est, learning_rate=0.01)
        else:
            bs[m] = RandomForestRegressor(random_state=0, n_estimators=n_est)
        bs[m].fit(X[Sm_idx], y[Sm_idx] - f_hat[Sm_idx])
        f_hat = f_hat + nu * bs[m].predict(X)
    # step 2
    lasso = Lasso(alpha = 0.1)
    Tx = np.empty([N, M])
    for i in range(M):
        Tx[:,i] = bs[i].predict(X)
    lasso.fit(Tx, y)
    return bs, lasso

In [56]:
def mse_isle(bs, lasso):
    M = len(bs)
    X_test, y_test = genXY(500, 100)
    Tx = np.empty([len(X_test), M])
    for i in range(M):
        Tx[:, i] = bs[i].predict(X_test)
    y_pred = lasso.predict(Tx)
    return np.mean((y_pred - y_test)**2)

In [52]:
bs, lasso = ISLE(X, y)

In [53]:
mse_isle(bs, lasso)

2.7838419536617893

In [37]:
from sklearn.linear_model import Lasso

In [38]:
lasso = Lasso(alpha=0.1)

In [39]:
lasso.fit(X,y)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [40]:
lasso.predict(X)

array([16.11963328, 15.90659572, 16.09956959, 15.80812887, 15.94646844,
       16.10173761, 16.18959057, 16.2608489 , 15.63376061, 15.53285881,
       16.11347873, 16.09418098, 15.74903257, 15.64689923, 16.19212549,
       16.07858233, 15.7814767 , 15.96743905, 15.93585248, 15.94528023,
       16.25631161, 15.55308676, 15.99240576, 15.98074183, 15.80278877,
       15.47398886, 15.71190252, 16.26247666, 15.85774815, 15.75119578,
       15.76947791, 15.68952973, 15.72934697, 16.01947211, 15.60091199,
       15.78829114, 15.71435376, 16.22997147, 16.11673847, 15.68506155,
       15.7420122 , 15.66014509, 15.40484804, 15.69442619, 15.80642629,
       15.97531893, 15.83585692, 15.9550448 , 15.78272366, 15.67924604,
       15.97635508, 16.00130144, 15.70796899, 15.45433448, 15.98993201,
       16.03347767, 16.04550161, 15.73203945, 15.50852142, 15.46085804,
       15.66386228, 15.90474803, 15.41145016, 15.81288178, 15.82982495,
       16.27315581, 16.08020934, 15.91556429, 15.48020588, 16.32